In [57]:
# Import modules
import torch

In [58]:
# Create a parallel db generator which removes one record from the original db and keeps the rest of the records
def par_db_generator(db, idx):
    return torch.cat((db[0:idx], db[idx+1:]))

In [59]:
# Backle up the parallel databases together
# it depends entirely on the length of the first db
def par_dbs(db):
    """This creates an array with the datase columns = len(db-1)
    Since we are removing one record at a time and returning the remaining values
    """
    db_all = []
    
    for i in range(len(db)):
        par_db = par_db_generator(db, i)
        db_all.append(par_db)
        
    return db_all

In [60]:
# Create a function to generate the required data based on a given sample of users or records
def generate_db(rec_no):
    """This function generates the db that is then usd to generate the parallel dbs"""
    db = torch.rand(rec_no)>0.5
    # print('Original DB\n{}'.format(db))
    
    #Generate the entire parallel db
    pdbs = par_dbs(db)
    
    return db, pdbs    

In [61]:
# -- TODO -- 
# Print the original database
db, pdb = generate_db(10)
db

tensor([0, 0, 0, 1, 0, 1, 1, 1, 1, 1], dtype=torch.uint8)

In [62]:
print(pdb)

[tensor([0, 0, 1, 0, 1, 1, 1, 1, 1], dtype=torch.uint8), tensor([0, 0, 1, 0, 1, 1, 1, 1, 1], dtype=torch.uint8), tensor([0, 0, 1, 0, 1, 1, 1, 1, 1], dtype=torch.uint8), tensor([0, 0, 0, 0, 1, 1, 1, 1, 1], dtype=torch.uint8), tensor([0, 0, 0, 1, 1, 1, 1, 1, 1], dtype=torch.uint8), tensor([0, 0, 0, 1, 0, 1, 1, 1, 1], dtype=torch.uint8), tensor([0, 0, 0, 1, 0, 1, 1, 1, 1], dtype=torch.uint8), tensor([0, 0, 0, 1, 0, 1, 1, 1, 1], dtype=torch.uint8), tensor([0, 0, 0, 1, 0, 1, 1, 1, 1], dtype=torch.uint8), tensor([0, 0, 0, 1, 0, 1, 1, 1, 1], dtype=torch.uint8)]


In [63]:
# -- TODO -- 
# Print the parallel databases
print(db[7])
pdb = pdb[7]

tensor(1, dtype=torch.uint8)


In [64]:
# -- TODO --
# print a single value to evaluate on
# This is to confirm whether or not this value has effect on the final outcome
# sum(db)
pdb


tensor([0, 0, 0, 1, 0, 1, 1, 1, 1], dtype=torch.uint8)

In [65]:
# Differencing attack using sum()
sum(db) - sum(pdb)

tensor(1, dtype=torch.uint8)

In [66]:
# Differencing attack using mean()
torch.abs(sum(db).float()/len(db) -sum(pdb).float()/len(pdb))

tensor(0.0444)

In [67]:
# Its Equivalent
# average of the binary data, i.e, either 0 or 1
# Divide by the number of samples

# --TODO--



In [70]:
# Differencing attack using threshold()
# check the sum of the db and then use the lower value of the sum as the threshold

# -- TODO --
sum(db)


tensor(6, dtype=torch.uint8)

In [71]:

(sum(db).float()>=6) - (sum(pdb).float()>6)

tensor(1, dtype=torch.uint8)